<a href="https://colab.research.google.com/github/namitasathish/Essential-Commodities-Price-Predictor-MODEL/blob/main/Essential_Commodities_Price_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import warnings

In [ ]:
pip install statsmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.9/233.9 kB 12.1 MB/s eta 0:00:00


In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
file_path = '/content/RS_Session_260_AU_1830_1.csv'
df = pd.read_csv(file_path)
print(df.head())
df.shape

   Sl. No.                      Food Commodities             Units  \
0        1  Annual Average Retail Prices of Rice  Unit - (Rs./Kg.)   
1        2  Annual Average Retail Prices of Rice  Unit - (Rs./Kg.)   
2        3  Annual Average Retail Prices of Rice  Unit - (Rs./Kg.)   
3        4  Annual Average Retail Prices of Rice  Unit - (Rs./Kg.)   
4        5  Annual Average Retail Prices of Rice  Unit - (Rs./Kg.)   

              State/UT   2018   2019   2020   2021   2022   2023  
0  Andaman and Nicobar  34.15  38.82  43.11  42.75  44.60  41.22  
1       Andhra Pradesh  31.63  35.11  41.72  46.13  48.65  51.52  
2    Arunachal Pradesh  29.59  31.95  32.51  25.79  28.57  33.00  
3                Assam  30.00  30.00  31.51  35.39  33.22  38.03  
4                Bihar  31.13  30.83  30.12  33.11  36.09  36.37  


(742, 10)

In [ ]:
if df.empty:
    raise ValueError("The dataset is empty. Please provide a valid dataset.")

In [ ]:
df.fillna(np.nan, inplace=True)
df = df.dropna()

In [ ]:
relevant_columns = ['Food Commodities', 'State/UT', '2018', '2019', '2020', '2021', '2022', '2023']
df = df[relevant_columns]

for year in ['2018', '2019', '2020', '2021', '2022', '2023']:
    df[year] = pd.to_numeric(df[year], errors='coerce')


In [ ]:
def forecast_price(commodity, state_ut, forecast_year):

    commodity_data = df[(df['Food Commodities'] == commodity) & (df['State/UT'] == state_ut)]

    if commodity_data.empty:
        raise ValueError(f"No data available for commodity: {commodity} in {state_ut}.")

    price_data = commodity_data.iloc[0, 2:].values
    price_data = pd.to_numeric(price_data, errors='coerce')

    if np.isnan(price_data).any():
        raise ValueError("The price data contains invalid or NaN values. Cannot proceed with forecasting.")

    model = ARIMA(price_data, order=(1, 1, 1))
    fitted_model = model.fit()
    forecast_steps = forecast_year - 2023
    if forecast_steps <= 0:
        raise ValueError("The forecast year must be in the future.")

    forecast_result = fitted_model.forecast(steps=forecast_steps)
    forecast_price = forecast_result[-1]
    return forecast_price


In [ ]:
commodity_name = input("Enter the name of the commodity (e.g., Annual Average Retail Prices of Rice): ")
state_ut_name = input("Enter the name of the state/UT (e.g., Andhra Pradesh): ")
year_to_predict = int(input("Enter the future year for prediction (2024 or later): "))

try:
    predicted_price = forecast_price(commodity_name, state_ut_name, year_to_predict)
    print(f"The predicted price for {commodity_name} in {state_ut_name} for the year {year_to_predict} is Rs.{predicted_price:.2f}/Kg.")
except ValueError as e:
    print(f"Error: {e}")

Enter the name of the commodity (e.g., Annual Average Retail Prices of Rice): Annual Average Retail Prices of Rice
Enter the name of the state/UT (e.g., Andhra Pradesh): Andhra Pradesh
Enter the future year for prediction (2024 or later): 2025
The predicted price for Annual Average Retail Prices of Rice in Andhra Pradesh for the year 2025 is Rs.54.95/Kg.
